In [1]:
import pyrogram
import pickle

dat = pickle.load(open('data/messages.pkl', 'rb'))

In [2]:
print(dat[3])


{
    "_": "Message",
    "id": 325907,
    "from_user": {
        "_": "User",
        "id": 227805,
        "is_self": false,
        "is_contact": false,
        "is_mutual_contact": false,
        "is_deleted": false,
        "is_bot": false,
        "is_verified": false,
        "is_restricted": false,
        "is_scam": false,
        "is_fake": false,
        "is_support": false,
        "is_premium": false,
        "first_name": "Oleksii",
        "status": "UserStatus.RECENTLY",
        "username": "alexzhv13",
        "dc_id": 2,
        "photo": {
            "_": "ChatPhoto",
            "small_file_id": "AQADAgADVccxG654oEsAEAIAA915AwAFqJeWoexnA3sABB4E",
            "small_photo_unique_id": "AgADVccxG654oEs",
            "big_file_id": "AQADAgADVccxG654oEsAEAMAA915AwAFqJeWoexnA3sABB4E",
            "big_photo_unique_id": "AgADVccxG654oEs"
        }
    },
    "date": "2024-09-22 10:50:48",
    "chat": {
        "_": "Chat",
        "id": -1001822567201,
        "type": "Ch

In [3]:
msg_tops = {}
tops = {}
msgs = {}
msgs_by_id = {}
msgs_quotes = {}
for i, d in enumerate(reversed(dat)):
    top = msg_tops.get(d.reply_to_message_id)
    if not top:
        msg_tops[d.reply_to_message_id] = d.reply_to_message_id
        top = d.reply_to_message_id
    msg_tops[d.id] = top
    msgs_by_id[d.id] = d
    tops.setdefault(top, [0, 0])
    msgs.setdefault(top, [])
    if d.reply_to_message_id:
        msgs_quotes.setdefault(d.reply_to_message_id, 0)
        msgs_quotes[d.reply_to_message_id] += 1

    if d.text:
        tops[top][0] += 1
        tops[top][1] += len(d.text)
        msgs[top].append(d)
    if 0:
        print(d.date, d.from_user.username, top, d.id, d.reply_to_message_id, d.text)
        if i > 10:
            break


In [4]:
t = list(sorted(tops.items(), key=lambda x: -x[1][0]))
t

[(92406, [32431, 2169905]),
 (4621, [5423, 361251]),
 (3389, [4291, 371518]),
 (5489, [3920, 307077]),
 (105418, [3785, 329030]),
 (26730, [2938, 231552]),
 (7626, [2694, 194551]),
 (2340, [2215, 178146]),
 (2776, [1594, 162379]),
 (4705, [655, 43829]),
 (2488, [610, 61006]),
 (25827, [569, 55870]),
 (2623, [533, 47695]),
 (4639, [509, 33378]),
 (1149, [280, 21072]),
 (7265, [181, 14294]),
 (19091, [160, 21207]),
 (4638, [150, 14610]),
 (319436, [145, 14163]),
 (27200, [143, 8621]),
 (308922, [98, 4699]),
 (2789, [59, 12084]),
 (2374, [24, 8722]),
 (299675, [24, 2528]),
 (303824, [23, 1353]),
 (255839, [21, 2844]),
 (1136, [20, 16412]),
 (299496, [19, 1180]),
 (246921, [18, 609]),
 (289085, [18, 672]),
 (272206, [17, 1583]),
 (306536, [17, 585]),
 (209471, [17, 1977]),
 (250981, [16, 862]),
 (272227, [16, 1642]),
 (258442, [14, 1222]),
 (264034, [14, 1084]),
 (265626, [14, 3028]),
 (290285, [14, 870]),
 (163391, [13, 1005]),
 (24747, [13, 781]),
 (277210, [13, 1556]),
 (293578, [13, 10

In [5]:
'''
[(92406, [32431, 2169905]), # Srachivnya
 (4621, [5423, 361251]),    # Itshechna
 (3389, [4291, 371518]),    # Pravo
 (5489, [3920, 307077]),    # Avto
 (105418, [3785, 329030]),  # Relocation
 (26730, [2938, 231552]),   # Consoli
 (7626, [2694, 194551]),    # Finances
 (2340, [2215, 178146]),    # Common Chat
 (2776, [1594, 162379]),    # Housing
 (4705, [655, 43829]),      # Culture
 (2488, [610, 61006]),      # Travel
 (25827, [569, 55870]),     # Job Search
 (2623, [533, 47695]),      # Medical
 (4639, [509, 33378]),      # Food
 (1149, [280, 21072]),      # Mental Health
 (7265, [181, 14294]),      # SecondHand
 (19091, [160, 21207]),     # Kids
 (4638, [150, 14610]),      # Pets
 (319436, [145, 14163]),    # Investments
 (27200, [143, 8621]),      # Memes
'''

thread_id = 92406
data = []
start = 500
cnt = 350
for i, m in enumerate(msgs[thread_id][start:start+cnt]):
    if m.from_user:
        user = m.from_user.username or f"{m.from_user.first_name}({m.from_user.id})"
        base = msgs_by_id.get(m.reply_to_message_id)
        msg = [f"== {m.date} {m.id} ==\n"]
        if base and base.text and base.from_user:
            base_user = base.from_user.username or f"{base.from_user.first_name}({base.from_user.id})"
            bt = base.text.splitlines()
            if len(bt) > 1:
                bt = f"{base_user}> " + "\n > ".join(bt)
            else:
                bt = f"{base_user}> " + bt[0]
            msg.append(f"{bt}\n")
        msg.append(f"{user}> {m.text}\n")
        if m.reactions and m.reactions.reactions:
            for r in m.reactions.reactions:
                msg.append(f"{r.emoji} {r.count} ")
            msg.append("\n")
        if m.id in msgs_quotes:
            msg.append(f"replies: {msgs_quotes[m.id]}\n")            
        msg.append("\n")
        data.append("".join(msg))
    if i == 0:
        print(m.date)
print(m.date)


2024-07-25 12:12:26
2024-07-25 19:53:33


In [6]:
print(data[4])

== 2024-07-25 12:19:25 250891 ==
Artem(667857019)> схема має бути в цілому українсько-російського формату: виділяють міліон євро організатору, він дає далі сто тисяч місцевому оргу, на місці орг за півтори тис євро збирає мітинг на 30 калєк




In [7]:
l = len("".join(data))
print(l, l/2.72)

62984 23155.882352941175


In [9]:
import openai
key=open("key.txt").read().strip()
client = openai.OpenAI(api_key=key)

In [10]:
data_txt = "".join(data)

sys_prompt = """
Task: Summarize Conversations from Telegram Channel

1. Generate a summary of the discussions within the channel.
2. Identify the top 10 specific conversations, do not use general terms like "taxes",
    "adaptation" and so on, use specific terms from chat.
    For each conversation:
      * List the main participants involved.
      * Provide a detailed summary in the original language.
      * include 5 most important quotes (include ({message_id} {username}) in quote)
      * Final conclusion
3. Tone Analysis: Note that many comments may carry a sarcastic tone. If it’s ambiguous whether a statement is sincere or not, interpret it as having an ironic or mocking tone.
4. Zrada vs. Peremoga (ZvP) Rating: Assign a ZVP rating for each topic on a scale from -5 (complete betrayal) to 5 (complete victory). Justify the assigned value with clear reasoning.
5. Overall Summary: Conclude with an overall sentiment analysis of the chat and its cumulative ZvP rating.
6. Participant Activity: Compile a list of the top participants along with:
  * A summary of their activity.
  * 3 top quotes
  * An individual ZvP rating for each participant, with explanations.
    
Common terms:
 ZUS - Poland goverment agency for social insurance
 PIT - income tax
 "ухилянт" - person who left Ukraine to avoid being mobilised
 many other terms are based on Polish to Ukrainian phonetic adaptation

People related context:
 * Most of participants of this chat are ukrainians living in Poland and working in software development related companies.
 * Some of them left Ukraine before war, but most people moved to Poland due to russian invasion.
 * Some people already moved from Poland to other countries, some people are planning this.

Notes:
 * Provide a comprehensive response to ensure all elements are thoroughly covered and avoid prematurely ending the message.
 * **Generate answer in Ukrainian language.**
 * Use informal playful tone.
 * Keep chat-specific slang terms as is.
 * Do not use term "sarcastic", "joke", "joking" too much, use appropriate replacement terms.
     Do not focus on this "joking" aspect to much, find some other points.
 * Focus on recent messages.
 * Generate a concise list of topics first as "content index" and final response after that.

Example output:
====
**Топ обговорення:**

1. Обговорення відео з Instagram
2. Дискусія про фінансові вимоги жінок
3. Обговорення кави та кав'ярень
4. Заклик до мобілізації українців за кордоном
5. Обговорення демографічної ситуації в Україні
6. Обговорення мусульман в Кракові та Закопанах)

---

**1. Обговорення відео з Instagram**

*Основні учасники:* hvala_bok, Artem, IllyaVolkov, andrii, is_lll, werq142

*Стислий опис:* Учасники обговорюють відео з Instagram, в якому жінка висловлює свої вимоги до потенційного партнера. Дискусія зосереджена на її прямолінійності та вимогах.

*Цитати:*
- "Яка прекрасна людина" (249976 Artem)
- "Ну якщо не рофл то і так молодець - чесно заявила свою позицію" (249982 is_lll)
- "Просто треба більше заробляти" (250000 hvala_bok)
- "А для чого тобі тоді таке чудо?" (249989 Oleksandr)
- "Ну відразу сказала що не для слабонєрвних" (249986 werq142)

*Висновок:* Учасники жартують про вимоги жінки, обговорюючи, чи є вони обґрунтованими.

*ZvP рейтинг:* -1 (легка зрада) - обговорення з іронічним тоном, що підкреслює нереалістичність вимог.

---

**2. Дискусія про фінансові вимоги жінок**

*Основні учасники:* mykytasahan7, jack_bauer_d, Oleksandr, Andrii99887

*Стислий опис:* Учасники обговорюють фінансові вимоги жінок, які очікують, що партнери покриватимуть їхні витрати. Дискусія переходить до обговорення ролі жінок у стосунках.

*Цитати:*
- "А що вона мені може дати і нащо вона така взагалі" (250064 Andrii99887)
- "Якшо вона тіпу зайде прям пздц - буде горіти нею, то він має підпадати під її умови" (250065 jack_bauer_d)
- "Просто треба більше заробляти" (250000 hvala_bok)
- "Ну різні ситуації є" (250055 Oleksandr)
- "Це як ракова пухлина, яка передається через Тік ток і інсті" (250054 Andrii99887)

*Висновок:* Учасники висловлюють скептицизм щодо таких вимог, підкреслюючи, що не всі жінки мають такі очікування.

*ZvP рейтинг:* -2 (зрада) - обговорення з негативним тоном щодо вимог жінок.

---

**3. Обговорення кави та кав'ярень**

*Основні учасники:* Artem, werq142, Konark96, Andrii99887

*Стислий опис:* Учасники обговорюють ціни на каву в Польщі, різні види кави та кавомашини. Дискусія зосереджена на якості кави та вартості.

*Цитати:*
- "Кавова шиза у польщі присутня у явній формі" (250183 Artem)
- "Часто капучино з автоматичної каво машини краще ніж те що роблять ці срані баристи" (250208 Andrii99887)
- "У мене є кавʼярня у домі, кава 16 плн" (250178 Artem)
- "В жабці по 6-8" (250192 werq142)
- "Лавацу взагалі не треба брати, є польскі паларні кави дуже добрі" (250227 Artem)

*Висновок:* Учасники обговорюють переваги кавомашин перед кав'ярнями, підкреслюючи високу вартість кави в Польщі.

*ZvP рейтинг:* 0 (нейтрально) - обговорення без особливого емоційного забарвлення.

---

**4. Заклик до мобілізації українців за кордоном**

*Основні учасники:* hvala_bok, Konark96, m_di_di, jack_bauer_d

*Стислий опис:* Обговорення заклику до мобілізації українців, які живуть за кордоном. Учасники висловлюють скептицизм та обговорюють можливі наслідки.

*Цитати:*
- "Готуємся посони, за сцикунів скоро візьмуться" (250318 hvala_bok)
- "Так він же сказав що зброї у рф більше в декілька разів, сенс тоді?" (250321 Konark96)
- "То ми вже до Аргентини приєднуємося" (250330 m_di_di)
- "Сирський закликав українців, які живуть за кордоном, взяти участь у війні" (250382 hvala_bok)
- "Ну і ясно що це не буде анкета в легіон на паспорт бо вже всі собі зробили" (250388 hvala_bok)

*Висновок:* Учасники висловлюють іронічний скептицизм щодо мобілізації, підкреслюючи, що багато хто вже має плани на еміграцію.

*ZvP рейтинг:* -3 (значна зрада) - обговорення з негативним тоном щодо мобілізації.

---

**5. Обговорення демографічної ситуації в Україні**

*Основні учасники:* Andrii99887, zaz1104

*Стислий опис:* Учасники обговорюють демографічну ситуацію в Україні, жартуючи про необхідність народжувати більше дітей.

*Цитати:*
- "Не народила 3 дітей не жінка" (250468 Andrii99887)
- "Мінімум 12" (250471 zaz1104)
- "Чоловіки вмирають, жінки народжують" (250478 Andrii99887)
- "Ідеальний світ" (250478 Andrii99887)
- "Тоді і 12 стрільне" (250477 zaz1104)

*Висновок:* Обговорення з іронічним тоном, підкреслюючи нереалістичність таких очікувань.

*ZvP рейтинг:* -1 (легка зрада) - обговорення з іронічним тоном.

---

**6. Обговорення мусульман в Кракові та Закопанах**

*Основні учасники:* igorzharii, olekoleko

*Стислий опис:* Учасники обговорюють велику кількість мусульман в Кракові та Закопанах, жартуючи про це.

*Цитати:*
- "Чуєте а хто з Кракова" (250503 igorzharii)
- "А чого у вас стільки мусульман/арабів" (250505 igorzharii)
- "Тут якась мусульманська мекка як для євреїв в Умані?" (250507 igorzharii)
- "З білорусі набігли" (250511 olekoleko)

*Висновок:* Обговорення з іронічним тоном, підкреслюючи несподіваність такої ситуації.

*ZvP рейтинг:* 0 (нейтрально) - обговорення без особливого емоційного забарвлення.

---

**Загальний підсумок:**

Загальний тон чату іронічний та скептичний, з акцентом на обговорення нереалістичних вимог та ситуацій. Учасники часто жартують і висловлюють скептицизм щодо різних тем, від мобілізації до фінансових вимог жінок.

**Загальний ZvP рейтинг:** -2 (зрада) - загальний негативний тон обговорень, з акцентом на скептицизм та іронію.

---

**Активність учасників:**

1. **Artem(667857019)**
   - Активність: Часто коментує різні теми, особливо обговорення кави та фінансових вимог.
   - Топ цитати:
     - "Кавова шиза у польщі присутня у явній формі" (250183)
     - "У мене є кавʼярня у домі, кава 16 плн" (250178)
     - "Яка прекрасна людина" (249976)
   - ZvP рейтинг: -1 (легка зрада) - часто іронізує та висловлює скептицизм.

2. **hvala_bok**
   - Активність: Ініціює обговорення мобілізації та відео з Instagram.
   - Топ цитати:
     - "Готуємся посони, за сцикунів скоро візьмуться" (250318)
     - "Просто треба більше заробляти" (250000)
     - "Сирський закликав українців, які живуть за кордоном, взяти участь у війні" (250382)
   - ZvP рейтинг: -2 (значна зрада) - висловлює скептицизм щодо мобілізації.

3. **Andrii99887**
   - Активність: Активно обговорює фінансові вимоги жінок та демографічну ситуацію.
   - Топ цитати:
     - "А що вона мені може дати і нащо вона така взагалі" (250064)
     - "Часто капучино з автоматичної каво машини краще ніж те що роблять ці срані баристи" (250208)
     - "Не народила 3 дітей не жінка" (250468)
   - ZvP рейтинг: -2 (значна зрада) - висловлює скептицизм та іронію щодо різних тем.

4. **Konark96**
   - Активність: Обговорює мобілізацію та каву.
   - Топ цитати:
     - "Так він же сказав що зброї у рф більше в декілька разів, сенс тоді?" (250321)
     - "В регіонах капучино 10-12" (250178)
     - "Ну я і у Варшаві знайду за 10" (250199)
   - ZvP рейтинг: -1 (легка зрада) - висловлює скептицизм щодо мобілізації та цін на каву.

5. **mykytasahan7**
   - Активність: Обговорює фінансові вимоги жінок та мобілізацію.
   - Топ цитати:
     - "да там пиздець" (250382)
     - "в рашці 150 млн" (250392)
     - "А що вона мені може дати і нащо вона така взагалі" (250064)
   - ZvP рейтинг: -2 (значна зрада) - висловлює скептицизм та іронію щодо різних тем.
===
"""

prompt = f"""This is a history of telegram chat:
===========
{data_txt}
===========
"""

# Call the API for chat completions
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",  # or use the model you prefer
    #model="gpt-4o-mini",  # or use the model you prefer
    messages=[{"role": "system", "content": sys_prompt},{"role": "user", "content": prompt}],
    temperature=0.3,
    max_completion_tokens=5000,
)


# Get the summary from the response
summary = response.choices[0].message.content
print(summary)

**Топ обговорення:**

1. Обговорення підтримки ЛНР/ДНР
2. Перереєстрація житла в Україні
3. Мобілізація українців за кордоном
4. Проблеми з документами та резидентством
5. Вплив політичних рішень на українців в ЄС
6. Обговорення ситуації в Молдові
7. НАТО та російські безпілотники
8. Історичні будівлі Києва
9. Плани на еміграцію
10. Робота та прокрастинація в IT

---

**1. Обговорення підтримки ЛНР/ДНР**

*Основні учасники:* VladyslavZotov, jack_bauer_d, Artem, hvala_bok, rogryyy

*Стислий опис:* Учасники обговорюють, чому деякі політики підтримують ЛНР/ДНР, висловлюючи свої припущення щодо фінансових мотивів.

*Цитати:*
- "не знав цього, цікаво, а яким чином вони за лнр днр, як аргументують?" (250887 VladyslavZotov)
- "за російський кошт купка головуючих пофіг як аргументує, решта корисні ідіоти" (250890 Artem)
- "схема має бути в цілому українсько-російського формату" (250891 Artem)
- "а поляки ж до речі так і не визначились чи це були рфні агенти чи ні?" (250889 rogryyy)
- "Ну може 

In [11]:
print(response.usage)

CompletionUsage(completion_tokens=4109, prompt_tokens=28890, total_tokens=32999, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))
